In [2]:
import numpy as np
import xarray as xr
import pandas as pd
import numcodecs
from dask.distributed import Client, progress, LocalCluster
import zarr
import glob
import netCDF4

In [3]:
cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:54803 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [4]:
# Invariants
year = 2002
prefix = '/Volumes/files/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1'

In [5]:
def generate_file_list(start_doy, end_doy):   
    """
    Given a start day and end end day, generate a list of file locations.
    Assumes a 'prefix' and 'year' variables have already been defined.
    'Prefix' should be a local directory or http url and path.
    'Year' should be a 4 digit year.
    """
    days_of_year = list(range(start_doy, end_doy))
    print(days_of_year)
    print("{}/{}/{}/*.nc".format(prefix, year, start_doy))
    fileObjs = []

    for doy in days_of_year:
        file = glob.glob("{}/{}/{}/*.nc".format(prefix, year, doy))[0]
        fileObjs.append(file)
    return fileObjs

In [8]:
%%time
start_doy = 152
end_doy = 157   
fileObjs = generate_file_list(start_doy, end_doy)
print("start doy: {}, file: {}".format(start_doy, fileObjs[0].split('/')[-1]))
print("end doy: {}, file: {}".format(end_doy, fileObjs[-1].split('/')[-1]))          
ds = xr.open_mfdataset(fileObjs, parallel=True, combine='by_coords')

[152, 153, 154, 155, 156]
/Volumes/files/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2002/152/*.nc
start doy: 152, file: 20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
end doy: 157, file: 20020605090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
CPU times: user 214 ms, sys: 37.8 ms, total: 252 ms
Wall time: 1.88 s
